In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import gensim

from wordcloud import WordCloud
from gensim import corpora
from gensim import models
from gensim.corpora.dictionary import Dictionary

from collections import defaultdict
from gensim.parsing.preprocessing import STOPWORDS

In [2]:
import string
import logging

In [3]:
dfname = 'FP4'

df1 = pd.read_pickle('dfs/' + dfname)
df = df1[['title','objective']]
df = df.dropna(how='any')
df['merged'] = df['title'] + ' ' + df['objective']

In [4]:
objectives = df['merged']
# objectives = pd.concat([ df['objective'] for df in [df4, df5, df6, df7, df20] ])

In [5]:
RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])

objectives = objectives.str.lower().str.replace('%l', '').str.replace(RE_PUNCTUATION, ' ')

In [6]:
objectives_split = objectives.str.strip().str.split()
objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if len(token) > 2])
objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if not(token.isdigit())])
objectives_split.head(2)

0                                       [spot, tation]
1    [formation, and, occurrence, nitrous, acd, the...
Name: merged, dtype: object

In [8]:
list_stopwords = ['data','will', 'develop', 'development', 'project', 'research', 'new', 'use', 'europe', 'european', 'based']
if dfname == 'FP4':
    list_stopwords.append('des')

additional_stopwords = set(list_stopwords)
stopwords = set(STOPWORDS) | additional_stopwords

objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if token not in stopwords])
objectives_split.head(2)

0                                       [spot, tation]
1    [formation, occurrence, nitrous, acd, atmosphe...
Name: merged, dtype: object

In [9]:
frequency = defaultdict(int)
for text in objectives_split:
    for token in text:
        frequency[token] += 1

objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if (frequency[token] > 5)])

objectives_dictionary = Dictionary(objectives_split)

In [10]:
class ObjectivesCorpus(corpora.textcorpus.TextCorpus):
    def get_texts(self):
        return iter(self.input)
    def __calc_corpus_size__(self):
        logging.info('Calculating corpus size')
        self.length = 0
        self.num_words = 0
        for doc in self.get_texts():
            self.length += 1
            self.num_words += len(doc)
    def __len__(self):
        """Define this so we can use `len(corpus)`"""
        if 'length' not in self.__dict__:
            self.__calc_corpus_size__()
        return self.length
    def __str__(self):
        if 'num_words' not in self.__dict__:
            self.__calc_corpus_size__()
        return (str(self.length) + ' documents, ' + str(self.num_words)
                + ' words')
            

objectives_corpus = ObjectivesCorpus(objectives_split)

In [11]:
lda = gensim.models.ldamodel.LdaModel(corpus = objectives_corpus, 
                                        id2word = objectives_dictionary, 
                                        num_topics = 10,
                                        passes=2,
                                        random_state = np.random.seed(12),
                                        iterations = 7000)

In [12]:
lda.show_topics()

[(0,
  u'0.016*"quality" + 0.013*"software" + 0.013*"process" + 0.010*"design" + 0.010*"systems" + 0.009*"control" + 0.008*"management" + 0.007*"improvement" + 0.007*"cost" + 0.006*"test"'),
 (1,
  u'0.013*"fish" + 0.013*"water" + 0.010*"plant" + 0.009*"species" + 0.009*"production" + 0.009*"chemical" + 0.008*"food" + 0.007*"biological" + 0.006*"control" + 0.006*"plants"'),
 (2,
  u'0.010*"study" + 0.010*"cell" + 0.009*"molecular" + 0.008*"genetic" + 0.008*"aquaculture" + 0.007*"control" + 0.007*"wind" + 0.007*"role" + 0.007*"cells" + 0.006*"neutron"'),
 (3,
  u'0.021*"information" + 0.011*"transport" + 0.009*"network" + 0.009*"health" + 0.008*"services" + 0.007*"support" + 0.007*"results" + 0.007*"public" + 0.006*"countries" + 0.006*"programme"'),
 (4,
  u'0.010*"concerted" + 0.010*"systems" + 0.010*"satellite" + 0.008*"building" + 0.007*"atm" + 0.006*"fisheries" + 0.006*"integrated" + 0.006*"farm" + 0.006*"digital" + 0.006*"video"'),
 (5,
  u'0.020*"high" + 0.010*"power" + 0.010*"ene

In [13]:
from __future__ import division
import graphlab as gl
import pyLDAvis
import pyLDAvis.graphlab

[WARNING] graphlab.product_key: Unable to write current GraphLab Create license to /home/marilia/.graphlab/config. Ensure that this user account                         has write permission to /home/marilia/.graphlab/config to save the license for offline use.


This non-commercial license of GraphLab Create for academic use is assigned to iliadi15@aueb.gr and will expire on June 01, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1496321908.log
INFO:graphlab.cython.cy_server:GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1496321908.log


In [14]:
pyLDAvis.enable_notebook()

In [15]:
import pyLDAvis.gensim

pyLDAvis.gensim.prepare(lda, objectives_corpus, objectives_dictionary)

/usr/local/lib/python2.7/dist-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      16.624899        1       1  0.136030 -0.039309
3      15.683525        1       2  0.132961  0.166265
9      11.387656        1       3  0.166929  0.053990
5       9.754030        1       4  0.072950 -0.120347
8       9.638773        1       5  0.059113 -0.209068
1       9.616042        1       6 -0.209667 -0.119803
7       8.859886        1       7 -0.023933  0.050467
2       8.210324        1       8 -0.192944 -0.010877
4       5.742353        1       9  0.058558  0.058372
6       4.482513        1      10 -0.199998  0.170309, topic_info=     Category         Freq            Term        Total  loglift  logprob
term                                                                     
1939  Default  1648.000000      production  1648.000000  30.0000  30.0000
1750  Default   844.000000        software   844.000000  29.0000  29.0000
2877  Default  1009.000000            food  1009.000000  28.0000  28.0000
6044  Default  1631.000000     information  1631.000000  27.0000  27.0000
1969  Default  1445.000000         network  1445.000000  26.0000  26.0000
1730  Default   976.000000       transport   976.000000  25.0000  25.0000
1456  Default  1555.000000            high  1555.000000  24.0000  24.0000
5243  Default   529.000000         disease   529.000000  23.0000  23.0000
5216  Default   828.000000   environmental   828.000000  22.0000  22.0000
6374  Default   529.000000            fish   529.000000  21.0000  21.0000
5987  Default   762.000000           water   762.000000  20.0000  20.0000
1451  Default  1348.000000         process  1348.000000  19.0000  19.0000
1005  Default   617.000000           plant   617.000000  18.0000  18.0000
4791  Default   822.000000        services   822.000000  17.0000  17.0000
4876  Default   438.000000             gas   438.000000  16.0000  16.0000
173   Default   666.000000         service   666.000000  15.0000  15.0000
4708  Default  1007.000000          energy  1007.000000  14.0000  14.0000
760   Default  1573.000000      technology  1573.000000  13.0000  13.0000
1646  Default   431.000000         genetic   431.000000  12.0000  12.0000
2327  Default   338.000000           genes   338.000000  11.0000  11.0000
4047  Default   716.000000        economic   716.000000  10.0000  10.0000
2831  Default   376.000000         species   376.000000   9.0000   9.0000
4205  Default   469.000000          policy   469.000000   8.0000   8.0000
6861  Default   379.000000      resistance   379.000000   7.0000   7.0000
160   Default   417.000000             atm   417.000000   6.0000   6.0000
6592  Default   406.000000         optical   406.000000   5.0000   5.0000
6741  Default  1042.000000        analysis  1042.000000   4.0000   4.0000
3218  Default   380.000000            cell   380.000000   3.0000   3.0000
1079  Default   450.000000       molecular   450.000000   2.0000   2.0000
3345  Default  1476.000000      management  1476.000000   1.0000   1.0000
...       ...          ...             ...          ...      ...      ...
5373  Topic10    33.101160          muscle    34.496701   3.0637  -6.2011
639   Topic10    30.629965          russia    31.891437   3.0646  -6.2787
3419  Topic10    70.751068        maritime    75.706060   3.0373  -5.4415
1380  Topic10    72.334261       logistics    81.167234   2.9898  -5.4193
6972  Topic10    31.807559           ships    33.284716   3.0596  -6.2409
3896  Topic10    92.498982       resistant   110.161886   2.9302  -5.1734
2754  Topic10    46.970920         pricing    53.345919   2.9777  -5.8511
4504  Topic10    94.354383            road   128.002718   2.8000  -5.1536
5243  Topic10   281.936603         disease   529.655921   2.4744  -4.0590
2327  Topic10   189.295628           genes   338.859502   2.5227  -4.4573
5498  Topic10   172.489938            risk   318.266858   2.4924  -4.5503
3258  Topic10    43.520904         starter    52.0878